# Wing Model


This Jupyter Notebook documents the workflow for developing and evaluating a machine learning model to predict aerodynamic coefficients (lift and drag) for a wing model. The notebook includes the following steps:

1. **Import Libraries**: Import necessary libraries and modules for data processing, model building, and evaluation.
2. **Set Parameters and Paths**: Define global parameters and paths for data and model storage.
3. **Check GPU Availability**: Verify if a GPU is available for model training.
4. **Data Preparation**: 
    - Downsample input data using average pooling.
    - Organize input data into a structured format.
    - Create custom dataset classes for wing data and simulation data.
5. **Model Definition**: 
    - Define convolutional neural network (CNN) models for predicting lift and drag coefficients.
    - Define radial basis function (RBF) network models for airfoil data refining the lift and drag coefficient values.
    - Load pre-trained model weights.
6. **Dataset Preparation**: 
    - Prepare the wing dataset by loading and processing CSV files.
    - Extract relevant features and target variables.
    - The CNN and RBF models are used to perdict airfol level lift and drag coefficients.
    - The airfoil-level values are then converted into wing-level values using empirical relations. 
7. **LSTM Model with Attention**: 
    - Define an LSTM model with attention mechanism for sequence prediction.
    - Set hyperparameters for model training.
8. **Data Normalization**: Normalize input and output data using MinMaxScaler.
9. **Train-Validation Split**: Split the dataset into training and validation sets.
10. **Model Training**: 
     - Train the LSTM model with attention on the training dataset.
     - Evaluate the model on the validation dataset.
11. **Loss Visualization**: Plot training and validation loss curves.
12. **Model Saving**: Save the trained model and scalers if required.
13. **Model Testing**: 
     - Load test data and make predictions using the trained model.
     - Compare predicted results with ground truth data.
14. **Evaluation**: 
     - Compute evaluation metrics such as MAPE, R² score, and relative L2 norm error.
     - Print and summarize evaluation results across all test cases.



In [ ]:
import os
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
# from torchviz import make_dot
from scipy.fftpack import fft, ifft
from scipy.interpolate import CubicSpline

import matplotlib.pyplot as plt

import joblib

In [ ]:
PLOT_RESULTS = False
SAVE_TRAINED_MODEL = False

project_path = '/mnt/e/eVTOL_model/eVTOL-VehicleModel/'
data_path = project_path + 'data/wing_data/'
model_path = project_path + 'trained_models/'


In [ ]:
import datetime
date = datetime.date.today()

In [ ]:
# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Downsampling using average pooling
"""
Downsample the input array to 35 elements using interpolation.
"""

def downsample_to_35(input_array):
    input_tensor = torch.tensor(input_array, dtype=torch.float32)
    
    # Reshape the input to be 1D (if it's not already)
    if input_tensor.dim() == 1:
        input_tensor = input_tensor.unsqueeze(0).unsqueeze(0)  # Shape (1, 1, original_length)
    elif input_tensor.dim() == 2:
        input_tensor = input_tensor.unsqueeze(0)  # Shape (1, original_channels, original_length)
    
    # Perform interpolation to downsample to 35 elements
    downsampled_tensor = F.interpolate(input_tensor, size=35, mode='linear', align_corners=True)
    
    # Remove the unnecessary dimensions to return a 1D tensor
    downsampled_array = downsampled_tensor.squeeze().numpy()
    
    return downsampled_array

"""
Create the dataset
Arange the input data in columns [x, y, alpha (AOA), Re, M]
"""
def organize_data(x_f, y_f, alphas):

    Elements = []

    # Loop through the polars
    for n_file in range(len(x_f)):
        x_temp = x_f[n_file]
        y_temp = y_f[n_file]
        alpha_temp = alphas[n_file]
        
        for j in range(len(alpha_temp)):
            batch = []
            # Loop through the coodrinates
            for i in range(len(x_temp)-1):
                element = np.array([x_temp[i], y_temp[i], x_temp[i+1], y_temp[i+1], alpha_temp[j]])
                batch.append(element)
            batch = np.array(batch)
            batch = batch.flatten()
            Elements.append(batch)

    Elements = np.array(Elements)

    return Elements

In [ ]:
"""
Element Spatial Convolutional Neural Network model
Number of convolutional layers - 4
Number of fully connected layers - 2
"""

# Model for lift coefficient
class ESCNN_Cl(nn.Module):
    def __init__(self):
        super(ESCNN_Cl, self).__init__()
        
        # Conv1: Assume 1D Convolution
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=200, kernel_size=5, stride=5)
        self.relu1 = nn.ReLU()

        #conv2
        self.conv2 = nn.Conv1d(in_channels=200, out_channels=200, kernel_size=1)
        self.relu2 = nn.ReLU()

        # Conv3
        self.conv3 = nn.Conv1d(in_channels=200, out_channels=100, kernel_size=1)
        self.relu3 = nn.ReLU()
        
        # Conv4
        self.conv4 = nn.Conv1d(in_channels=100, out_channels=1, kernel_size=5, padding=2)
        self.relu4 = nn.ReLU()
        
        # Final fully connected layer to output scalar
        self.fc1 = nn.Linear(in_features=34, out_features=34)
        self.relu5 = nn.ReLU()

        self.fc2 = nn.Linear(in_features=34, out_features=1)
    
    def forward(self, x):
        # Reshape input if necessary, ensure it's in the shape (batch_size, channels, elements)
        x = x.view(-1, 1, 170)  # Reshape to (batch_size, channel=1, elements=170)
        
        x = self.conv1(x)  
        x = self.relu1(x)
        
        x = self.conv2(x)  
        x = self.relu2(x)

        x = self.conv3(x)  
        x = self.relu3(x)
        
        x = self.conv4(x)  
        x = self.relu4(x)
        
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)  
        x = self.relu5(x)

        x = self.fc2(x)
        
        return x


In [ ]:
"""
Element Spatial Convolutional Neural Network model
Number of convolutional layers - 3
Number of fully connected layers - 2
"""

class ESCNN_Cd(nn.Module):
    def __init__(self):
        super(ESCNN_Cd, self).__init__()
        
        # Conv1: Assume 1D Convolution
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=200, kernel_size=5, stride=5)
        self.relu1 = nn.ReLU()

        #conv2
        self.conv2 = nn.Conv1d(in_channels=200, out_channels=200, kernel_size=1)
        self.relu2 = nn.ReLU()
        
        # Conv2
        self.conv3 = nn.Conv1d(in_channels=200, out_channels=1, kernel_size=5, padding=2)
        self.relu3 = nn.ReLU()
        
        # Final fully connected layer to output scalar
        self.fc1 = nn.Linear(in_features=34, out_features=34)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(in_features=34, out_features=1)

        # Define learnable parameters for C_d0 and k
        self.Cd0 = nn.Parameter(torch.tensor(0.02))  # Initialized zero-lift drag coefficient
        self.k = nn.Parameter(torch.tensor(0.05))    # Initialized induced drag factor

    
    def forward(self, x):
        # Reshape input if necessary, ensure it's in the shape (batch_size, channels, elements)
        x = x.view(-1, 1, 170)  # Reshape to (batch_size, channel=1, elements=170)
        
        x = self.conv1(x)  
        x = self.relu1(x)
        
        x = self.conv2(x)  
        x = self.relu2(x)

        x = self.conv3(x)  
        x = self.relu3(x)
        
        
        x = torch.flatten(x, 1)
        
        x = self.fc1(x) 
        x = self.relu4(x)

        x = self.fc2(x)
        
        return x

In [ ]:
# Initialize airfoil model (Use the saved trained model)

# Cl Model
class RBFLayer_cl(nn.Module):
    def __init__(self, in_features, out_features, centers=None):
        super(RBFLayer_cl, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        # Centers initialized using KMeans or passed explicitly
        if centers is None:
            self.centers = nn.Parameter(torch.Tensor(out_features, in_features))
            nn.init.uniform_(self.centers, -1, 1)  # Random if not initialized with KMeans
        else:
            self.centers = nn.Parameter(torch.Tensor(centers))  # Set centers from KMeans
        
        # Initialize the beta (width) parameter, positive constraint with softplus
        self.log_beta = nn.Parameter(torch.ones(out_features) * torch.log(torch.tensor(0.1)))

    @property
    def beta(self):
        # Ensures beta is positive using softplus
        return F.softplus(self.log_beta)
    
    def forward(self, input):
        # Compute distances between inputs and centers
        x = input.unsqueeze(1).expand(-1, self.out_features, self.in_features)
        c = self.centers.unsqueeze(0).expand(input.size(0), -1, -1)
        
        # Squared Euclidean distance
        distances = torch.sum((x - c) ** 2, dim=-1).to(device)
        
        # Apply Gaussian RBF
        return torch.exp(-self.beta.unsqueeze(0) * distances)

class RBFNet_cl(nn.Module):
    def __init__(self, input_size, rbf_units, output_size, centers=None):
        super(RBFNet_cl, self).__init__()
        self.rbf = RBFLayer_cl(input_size, rbf_units, centers)
        self.fc = nn.Linear(rbf_units, output_size)
        
        # Initialize weights for the linear layer
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)
        
    def forward(self, x):
        rbf_out = self.rbf(x)
        return self.fc(rbf_out)

# Cd Model
class RBFLayer_cd(nn.Module):
    def __init__(self, in_features, out_features, centers=None):
        super(RBFLayer_cd, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        # Centers initialized using KMeans or passed explicitly
        if centers is None:
            self.centers = nn.Parameter(torch.Tensor(out_features, in_features))
            nn.init.uniform_(self.centers, -1, 1)  # Random if not initialized with KMeans
        else:
            self.centers = nn.Parameter(torch.Tensor(centers))  # Set centers from KMeans
        
        # Initialize the beta (width) parameter, positive constraint with softplus
        self.log_beta = nn.Parameter(torch.ones(out_features) * torch.log(torch.tensor(18.2)))

    @property
    def beta(self):
        # Ensures beta is positive using softplus
        return F.softplus(self.log_beta)
    
    def forward(self, input):
        # Compute distances between inputs and centers
        x = input.unsqueeze(1).expand(-1, self.out_features, self.in_features)
        c = self.centers.unsqueeze(0).expand(input.size(0), -1, -1)
        
        # Squared Euclidean distance
        distances = torch.sum((x - c) ** 2, dim=-1).to(device)
        
        # Apply Gaussian RBF
        return torch.exp(-self.beta.unsqueeze(0) * distances)

class RBFNet_cd(nn.Module):
    def __init__(self, input_size, rbf_units, output_size, centers=None):
        super(RBFNet_cd, self).__init__()
        self.rbf = RBFLayer_cd(input_size, rbf_units, centers)
        self.fc = nn.Linear(rbf_units, output_size)
        
        # Initialize weights for the linear layer
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)
        
    def forward(self, x):
        rbf_out = self.rbf(x)
        return self.fc(rbf_out)
     


In [ ]:
# Load the model weights
af_model_ESCNN_Cl = ESCNN_Cl()
af_model_ESCNN_Cl.load_state_dict(torch.load(model_path+'/models/airfoil/2024-11-18_model_Cl_ESCNN_lr1e-05_e1500_rbf170_convL4.pth'))
af_model_ESCNN_Cl = af_model_ESCNN_Cl.to(device)
af_model_ESCNN_Cl.eval()

# Load the model weights
af_model_ESCNN_Cd = ESCNN_Cd()
af_model_ESCNN_Cd.load_state_dict(torch.load(model_path+'/models/airfoil/2024-11-18_model_Cd_ESCNN_lr5e-05_e250_convL3.pth'))
af_model_ESCNN_Cd = af_model_ESCNN_Cd.to(device)
af_model_ESCNN_Cd.eval()



input_size = 140
output_size = 4
num_rbf_units = 4

kmeans_center_cl = torch.load(model_path+'/models/airfoil/2024-11-19_airfoil_model_Cl_ESCNN_RBF_lr0.06_epoch200_rbfUnits4_RBFcenters.pth')
kmeans_center_cd = torch.load(model_path+'/models/airfoil/2024-11-19_airfoil_model_Cd_ESCNN_RBF_lr0.025_epoch100_rbfUnits4_RBFcenters.pth')


class airfoilModel_cl(RBFNet_cl):
    def __init__(self):
        super(airfoilModel_cl, self).__init__(input_size, num_rbf_units, output_size, kmeans_center_cl)


class airfoilModel_cd(RBFNet_cd):
    def __init__(self):
        super(airfoilModel_cd, self).__init__(input_size, num_rbf_units, output_size, kmeans_center_cd)


# Initialize the model
airfoil_cl = airfoilModel_cl()
airfoil_cl.load_state_dict(torch.load(model_path+'/models/airfoil/2024-11-19_airfoil_model_Cl_ESCNN_RBF_lr0.06_epoch200_rbfUnits4.pth'))

airfoil_cd = airfoilModel_cd()
airfoil_cd.load_state_dict(torch.load(model_path+'/models/airfoil/2024-11-19_airfoil_model_Cd_ESCNN_RBF_lr0.025_epoch100_rbfUnits4.pth'))

airfoil_cl = airfoil_cl.to(device)
airfoil_cd = airfoil_cd.to(device)

airfoil_cl.eval()
airfoil_cd.eval()

In [ ]:
# Prepare wing dataset

# Condition function to filter subdirectories
def subdir_condition(subdir_name):
    """
    Condition: Only process subdirectories whose names start with 'propeller-example_dji'.
    Modify this function to apply a specific filtering logic.
    """
    return subdir_name.startswith('wing_dataset')  # Change this condition as needed


class WingDataset(Dataset):
    def __init__(self, root_dir, af_data_path='/mnt/e/Course_Materials/ROM/wing_model/wing_section/small_database_testing_csv', subdir_condition=None):
        """
        Args:
            root_dir (string): Root directory with subdirectories containing CSV files.
            af_data_path (string): Airfoil polar database directory contaiing airfoil polars of used airfoils.            
            subdir_condition (callable, optional): A function or condition to filter subdirectories by name.
        """
        self.root_dir = root_dir
        self.data = []
        self.targets = []
        self.time_data = []  # Store time data separately
        self.AOA_data = []
        self.v_inf_data = []
        self.cl_data = []
        self.cd_data = []

        self.cl_data = []
        self.cd_data = []
        self.fft_cl_r = []
        self.fft_cl_i = []
        self.fft_cd_r = []
        self.fft_cd_i = []
        
        self.subdir_condition = subdir_condition
        self.af_data_path = af_data_path

        # Traverse the root directory to gather data
        self._load_data()

    def _load_data(self):
        """
        Helper function to read CSV files from each subdirectory and extract relevant columns.
        """
        # Iterate through each subdirectory in the root directory
        for subdir, _, files in os.walk(self.root_dir):
            subdir_name = os.path.basename(subdir)
            
            # Apply subdirectory name condition
            if self.subdir_condition and not self.subdir_condition(subdir_name):
                continue

            for file in files:

                if file.endswith("_convergence.csv"):
                    # Load the CSV file
                    csv_path = os.path.join(subdir, file)
                    df = pd.read_csv(csv_path)
                    
                    # Extract necessary columns for input features from wing_convergence.csv
                    time = df['T'].values  # Time
                    
                    v_inf = df['Vinf'].values
                    AOA = df['alpha(eff)'].values
                    theta = df['theta'].values
                    vz = df['vVehicle_z'].values
                                        
                    # Extract Cl and Cq for output variables
                    cl = df['CL'].values  # Lift coefficient (CL)
                    cd = df['CD'].values  # Drag coefficient (CD)

                    fft_cl = fft(cl)
                    fft_cl_real = np.real(fft_cl)
                    fft_cl_imag = np.imag(fft_cl)

                    fft_cd = fft(cd)
                    fft_cd_real = np.real(fft_cd)
                    fft_cd_imag = np.imag(fft_cd)

                    # Add the geometry parameters
                    geom_data_path = os.path.join(subdir, 'wing_geometry_data.csv')         # Read from wing_geometry_data.csv file - b, ar, tr, sweep, dihedral
                    geom_df = pd.read_csv(geom_data_path, nrows=1)

                    ones_empty = np.ones_like(time)                                         # Empty array of ones
                    
                    b_data = geom_df["b_wing"].values
                    b_data_array = geom_df["b_wing"].values * ones_empty                    # Making this an array simplifies things later
                    
                    ar_data = geom_df["ar_wing"].values
                    ar_data_array = geom_df["ar_wing"].values * ones_empty
                    
                    tr_data = geom_df["tr"].values
                    tr_data_array = geom_df["tr"].values * ones_empty

                    sweep_data = geom_df["lambda"].values
                    sweep_data_array = geom_df["lambda"].values * ones_empty
                    
                    gamma_data = geom_df["gamma"].values
                    gamma_data_array = geom_df["gamma"].values * ones_empty
                    
                    # Store the following in separate lists for easy access
                    self.time_data.append(time)
                    self.AOA_data.append(AOA)
                    self.v_inf_data.append(v_inf)
                    
                    self.cl_data.append(cl)
                    self.cd_data.append(cd)

                    self.fft_cl_r.append(fft_cl_real)
                    self.fft_cl_i.append(fft_cl_imag)
                    self.fft_cd_r.append(fft_cd_real)
                    self.fft_cd_i.append(fft_cd_imag)

                    # Extract the airfoil details
                    af_name = geom_df["airfoil "].values
                    extension = '.csv'
                    af_name = str(af_name[0]+extension)

                    af_name_new = af_name.split('-')
                    af_coordinate = str(af_name_new[1]+'_coordinates.dat')          # Name of airfoil coordinates. Adjust it according to the names being used
                    # print(af_coordinate)
                    
                    af_polar_data = pd.read_csv(os.path.join(self.af_data_path, af_name), skiprows=10)
                    # af_polar_data = af_polar_data[[(af_polar_data["Alpha"] >= -2) & (af_polar_data["Alpha"] <= 12)]]
                    # af_coordinate_data = pd.read_csv(os.path.join(self.af_data_path, af_coordinate), delim_whitespace=True)  # or use delimiter=','
                    af_coordinate_data = np.loadtxt(os.path.join(self.af_data_path, af_coordinate))

                    af_coordinate_x = af_coordinate_data[:,0]
                    af_coordinate_y = af_coordinate_data[:,1] 

                    AOA_af_polar = af_polar_data["Alpha"].values
                    cl_af_polar = af_polar_data["Cl"].values
                    cd_af_polar = af_polar_data["Cd"].values

                    # Fit a polynomial to Cl and Cd data
                    # Create cubic spline interpolation
                    spline_cl_airfoil = CubicSpline(AOA_af_polar, cl_af_polar)
                    spline_cd_airfoil = CubicSpline(AOA_af_polar, cd_af_polar)

                    cl_poly_coeff = np.polyfit(AOA_af_polar, cl_af_polar, deg=6)
                    cd_poly_coeff = np.polyfit(AOA_af_polar, cd_af_polar, deg=6)

                    cl_airfoil_calc = spline_cl_airfoil(AOA)
                    cd_airfoil_calc = spline_cd_airfoil(AOA)

                    #-------------------------------------------------------------------------------------------------------------------
                    # Note - To determine the airfoil aerodynamic coefficients, there are two approaaches.
                    # 1. Use the polar file from the database to fit a polynomial function and then estimate the Cl/Cd for any given AOA.
                    # 2. USe the pre-trained Neural Network to predict the Cl/Cd.  
                    #-------------------------------------------------------------------------------------------------------------------
                    
                    # Using approach 2 - Using RBF NN to predict the Cl and Cd
                    #-------------------------------------------------------------------------------------------------------------------
                
                    degree = 3
                    
                    input_sequence_cl_NN = []
                    input_sequence_cd_NN = []

                    # Prepare the data to input to ESCNN Model
                    x_escnn = np.array(downsample_to_35(af_coordinate_x)).reshape(1, -1)
                    y_escnn = np.array(downsample_to_35(af_coordinate_y)).reshape(1, -1)
                    aoa_escnn = np.array(AOA_af_polar).reshape(1, -1)

                    elements_ESCNN = organize_data(x_escnn, y_escnn, aoa_escnn)

                    if elements_ESCNN.shape != (0,):
                        input_escnn = elements_ESCNN

                        input_escnn = torch.tensor(input_escnn, dtype=torch.float32).to(device)

                        # Evaluate the model on test dataset
                        with torch.no_grad():
                            Cl_escnn_pred = af_model_ESCNN_Cl.forward(input_escnn)
                            Cd_escnn_pred = af_model_ESCNN_Cd.forward(input_escnn)
                            
                        Cl_escnn_pred = Cl_escnn_pred.cpu().detach().numpy()  # Convert tensor to numpy array
                        Cl_escnn_pred = Cl_escnn_pred.squeeze(1)

                        Cd_escnn_pred = Cd_escnn_pred.cpu().detach().numpy()  # Convert tensor to numpy array
                        Cd_escnn_pred = Cd_escnn_pred.squeeze(1)

                        plt_af_polar_comparison = False     # If needed for debugging
                        if plt_af_polar_comparison == True:
                            plt.figure()
                            # plt.plot(aoa_test[j], Cl_escnn_pred)
                            # # plt.plot(alphas_t[0], Cl_eval_org_scale)
                            # plt.plot(aoa_test[j], cl_test[j])

                            plt.legend(['NN Model - ESCNN', 'UIUC database'])
                            # plt.title(r'$C_d$ vs $\alpha$ for {} airfoil'.format(keyword))
                            plt.xlabel(r'AOA [$\alpha$]')
                            plt.ylabel(r'$C_l$')

                    else:
                        continue

                    af_coordinate_x_i = downsample_to_35(af_coordinate_x)
                    af_coordinate_y_i = downsample_to_35(af_coordinate_y)
                    AOA_af_polar_i = downsample_to_35(AOA_af_polar)
                    cl_af_polar_i = downsample_to_35(Cl_escnn_pred)
                    cd_af_polar_i = downsample_to_35(Cd_escnn_pred)

                    input_sequence_cl_NN = [
                                            af_coordinate_x_i, af_coordinate_y_i, AOA_af_polar_i, cl_af_polar_i
                                        ]
                    
                    input_sequence_cd_NN = [
                                            af_coordinate_x_i, af_coordinate_y_i, AOA_af_polar_i, cd_af_polar_i
                                        ]



                    input_sequence_cl_NN = np.array(input_sequence_cl_NN, dtype=float).reshape(1, -1)
                    input_sequence_cd_NN = np.array(input_sequence_cd_NN, dtype=float).reshape(1, -1)
                    
                    # print("Airfoil Cl NN - Input shape: ", input_sequence_cl_NN.shape)
                    # print("Airfoil Cd NN - Input shape: ", input_sequence_cd_NN.shape)

                    NN_cl_model_ip_data = torch.tensor(input_sequence_cl_NN, dtype=torch.float32).to(device) 
                    NN_cd_model_ip_data = torch.tensor(input_sequence_cd_NN, dtype=torch.float32).to(device) 

            
                    # Neural network model to predict the airfoil aerodynamic coeficients    
                    with torch.no_grad():  # Disable gradient computation for inference
                        predicted_coefficients_cl = airfoil_cl(NN_cl_model_ip_data)
                        predicted_coefficients_cd = airfoil_cd(NN_cd_model_ip_data)

                    predicted_af_coefficients_cl = predicted_coefficients_cl.cpu().detach().numpy()
                    predicted_af_coefficients_cd = predicted_coefficients_cd.cpu().detach().numpy()
                    
                    # print(predicted_af_coefficients_cl)

                    # polynomial_cl_new = np.poly1d(predicted_coefficients[0])
                    polynomial_cl_pred = np.poly1d(predicted_af_coefficients_cl[0])
                    polynomial_cd_pred = np.poly1d(predicted_af_coefficients_cd[0])

                    x_new = np.linspace(AOA_af_polar[0], AOA_af_polar[-1], 100)
                    # y_new_cl = polynomial_cl_new(x_new_cl)
                    y_new_cl = polynomial_cl_pred(x_new)
                    y_new_cd = polynomial_cd_pred(x_new)

                    

                    # # plt.figure()
                    plt_af_polar_NN = False
                    if plt_af_polar_NN == True:
                        plt.figure(figsize=(15, 5))
                        # plt.title(af_name)
                        
                        plt.subplot(1,2,1)
                        plt.plot(AOA_af_polar, cl_af_polar, color='red', label='UIUC Database')
                        plt.plot(x_new, y_new_cl, label=f'Polynomial Degree {degree} - NN')
                        plt.xlabel(r'$\alpha$')
                        plt.ylabel(r'$C_l$')
                        plt.title(af_name)
                        plt.legend()


                        plt.subplot(1,2,2)
                        plt.plot(AOA_af_polar, cd_af_polar, color='red', label='UIUC Database')
                        plt.plot(x_new, y_new_cd, label=f'Polynomial Degree {degree} - NN')
                        plt.xlabel(r'$\alpha$')
                        plt.ylabel(r'$C_d$')
                        plt.title(af_name)
                        plt.legend()
                    
                    
                    cl_af_NN = polynomial_cl_pred(AOA)
                    cd_af_NN = polynomial_cd_pred(AOA)


                    # print(cl_af_NN)
                    # print(cl_airfoil_calc)
                    plt_af_lvl_coeff = False
                    if plt_af_lvl_coeff==True:
                        plt.figure(figsize=(15, 5))
                        
                        
                        plt.subplot(1,2,1)
                        plt.plot(time, cl_airfoil_calc, color='red', label='UIUC Database')
                        plt.plot(time, cl_af_NN, label=f'Polynomial Degree {degree} - NN')
                        plt.xlabel(r'$\alpha$')
                        plt.ylabel(r'$C_l$')
                        plt.title(af_name)
                        plt.legend()

                        plt.subplot(1,2,2)
                        plt.plot(time, cd_airfoil_calc, color='red', label='UIUC Database')
                        plt.plot(time, cd_af_NN, label=f'Polynomial Degree {degree} - NN')
                        plt.xlabel(r'$\alpha$')
                        plt.ylabel(r'$C_d$')
                        plt.title(af_name)
                        plt.legend()
                    
                    #-------------------------------------------------------------------------------------------------------------------
                    # Calculate the Cl and Cd using the emperical equations using airfoil aerodynamic coefficients
                    # Calculate Wing Aerodynamic Coefficients

                    vel_comp_factor_cl = 0.01                # Assumed from trial & error. Depends on flow velocity and wing geometry
                    vel_comp_factor_cd = 0.00005              # Assumed from trial & error. Depends on flow velocity and wing geometry
                    e = 0.85                                # Oswald factor

                    # cl_wing_calc = cl_airfoil_calc / (1 + cl_airfoil_calc / (np.pi * ar_data * e)) * vel_comp_factor_cl

                    # cd_induced = (cl_wing_calc ** 2) / (np.pi * ar_data * e) * vel_comp_factor_cd

                    cl_wing_calc = cl_af_NN / (1 + cl_af_NN / (np.pi * ar_data * e)) * vel_comp_factor_cl

                    cd_induced = (cl_wing_calc ** 2) / (np.pi * ar_data * e) * vel_comp_factor_cd

                    # Step 5: Total drag coefficient for the wing
                    cd_wing_calc = cd_af_NN + cd_induced
                    # cd_wing_calc = cd_airfoil_calc + cd_induced

                    # For each simulation, the input sequence is structured as (n_timesteps, n_features)
                    sequence_inputs = []
                    sequence_outputs = []
                    for i in range(len(time)):
                        input_data = [
                            time[i], AOA[i], v_inf[i], b_data_array[i], ar_data_array[i], tr_data_array[i], sweep_data_array[i], 
                            gamma_data_array[i], cl_wing_calc[i], cd_wing_calc[i]
                        ]
                        output_data = [cl[i], cd[i]]
                        
                        
                        sequence_inputs.append(input_data)
                        sequence_outputs.append(output_data)

                    sequence_inputs = np.array([sequence_inputs], dtype=float)
                    sequence_outputs = np.array([sequence_outputs], dtype=float)

                    # Append input sequence (n_timesteps, num_features) and output (Cl, Cd)
                    self.data.append(sequence_inputs)
                    self.targets.append(sequence_outputs)  # Append the whole Cl and Cd sequences


    def __len__(self):
        """
        Returns the total number of sequences in the dataset.
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Returns a single sequence and its targets.
        """
        inputs = self.data[idx]  # Input sequence: (n_timesteps, n_features)
        targets = self.targets[idx]  # Output: (n_timesteps, 2)
        # return inputs, targets
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)

    def get_variable(self, variable_name):
        """
        Returns a list of arrays for the specified variable.
        Args:
            'time' - timesteps
            
        """
        if variable_name == 'time':
            return self.time_data  # Return all time steps for each simulation
        elif variable_name == 'CL':
            return self.cl_data  # Return all omega (RPM) values for each simulation
        elif variable_name == 'CD':
            return self.cd_data
        elif variable_name == 'AOA':
            return self.AOA_data
        elif variable_name == 'Vinf':
            return self.v_inf_data  
        elif variable_name == 'fft_cl':
            return self.fft_cl_r, self.fft_cl_i 
        elif variable_name == 'fft_cd':
            return self.fft_cd_r, self.fft_cd_i 
        else:
            raise ValueError(f"Variable {variable_name} not supported.")

In [ ]:
# Define the custom dataset class
# Creates training and validation datasets
class SimulationDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx], dtype=torch.float32), torch.tensor(self.outputs[idx], dtype=torch.float32)

In [ ]:
root_train = data_path + '/training_data/'

# Wing geometry
b               = 2.489                     # (m) span length
ar              = 5.0                       # Aspect ratio b/c_tip
tr              = 1.0                       # Taper ratio c_tip/c_root
twist_root      = 0.0                       # (deg) twist at root
twist_tip       = 0.0                       # (deg) twist at tip
sweep          = 45.0                       # (deg) sweep
gamma           = 0.0                       # (deg) dihedral

dataset = WingDataset(root_train, subdir_condition=subdir_condition)
inputs, outputs = dataset[0:]

input_tensor = inputs.squeeze(1)  # Reshaping
print("Input shape:", input_tensor.shape)  # Should print: torch.Size([6, 145, 7])

output_tensor = outputs.squeeze(1)
print("Output shape:",output_tensor.shape)  # Should print: torch.Size([6, 145, 2])

In [ ]:
class LSTMNetWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads, bias=0.0):
        super(LSTMNetWithAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)

        # Multi-head attention layer
        self.multihead_attention = nn.MultiheadAttention(hidden_size, num_heads, batch_first=True)

        # Fully connected output layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        """
        x: Input tensor of shape (batch_size, seq_length, input_size)
        """
        # Pass through LSTM
        lstm_out, _ = self.lstm(x)  # lstm_out: (batch_size, seq_length, hidden_size)

        # Multi-head attention: Query, Key, and Value are all lstm_out
        attention_output, attention_weights = self.multihead_attention(lstm_out, lstm_out, lstm_out)

        # Pass attention output through the fully connected layer
        output = self.fc(attention_output)  # (batch_size, seq_length, output_size)

        # Add bias to the output
        output = output + self.bias

        return output, attention_weights

In [ ]:
# Hyperparameters
input_size = 10     # Number of input features (e.g., 7 input variables)
hidden_size = 50    # Number of LSTM units (hidden state size)
output_size = 2     # Number of output features (e.g., 2 for thrust and torque coefficients)
num_layers = 2      # Number of LSTM layers
learning_rate = 5e-4
num_epochs = 2000
batch_size = 8


In [ ]:
# Initialize scalers for input and output
input_scaler = MinMaxScaler()
output_scaler = MinMaxScaler()

# Reshape data for scaling (flatten along the time dimension)
inputs_reshaped = input_tensor.reshape(-1, input_size)
outputs_reshaped = output_tensor.reshape(-1, output_size)

print(inputs_reshaped.shape)
print(outputs_reshaped.shape)

# Fit and transform inputs and outputs
inputs_normalized = input_scaler.fit_transform(inputs_reshaped).reshape(input_tensor.shape)
outputs_normalized = output_scaler.fit_transform(outputs_reshaped).reshape(output_tensor.shape)

print("Normalized input shape:", inputs_normalized.shape)
print("Normalized output shape:", outputs_normalized.shape)


In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset randomly
inputs_train, inputs_val, outputs_train, outputs_val = train_test_split(
    inputs_normalized, outputs_normalized, test_size=0.25, random_state=42, shuffle=True
)

# Create datasets
train_dataset = SimulationDataset(inputs_train, outputs_train)
val_dataset = SimulationDataset(inputs_val, outputs_val)

# Create DataLoaders
trainDataLoader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  
valDataLoader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)  # No need to shuffle validation

In [ ]:
# Instantiate model, loss function, and optimizer

num_heads = 2
bias = 0.1
model = LSTMNetWithAttention(input_size, hidden_size, output_size, num_layers, num_heads, bias).to(device)

criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

## Train the model

In [ ]:


# Lists to store losses
train_losses = []
eval_losses = []

print("[INFO] training the network...")

# Training Loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, targets in trainDataLoader:
        inputs, targets = inputs.squeeze(1).to(device), targets.squeeze(1).to(device)

        # Forward pass
        outputs, attention_weights = model(inputs)
        
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    avg_train_loss = running_loss / len(trainDataLoader)
    train_losses.append(avg_train_loss)


    # Evaluation Loop
    model.eval()
    eval_loss = 0.0

    with torch.no_grad():
        for inputs, targets in valDataLoader:
            inputs, targets = inputs.squeeze(1).to(device), targets.squeeze(1).to(device)
            
            # Forward pass
            outputs, attention_weights = model(inputs)

            loss = criterion(outputs, targets)

            eval_loss += loss.item()

        avg_eval_loss = eval_loss / len(valDataLoader)
        eval_losses.append(avg_eval_loss)
    
    print(f'Epoch [{epoch+1}/{num_epochs}],\n Training Loss: {running_loss/len(trainDataLoader):.4f}')
    print(f'Evaluation Loss: {eval_loss/len(valDataLoader):.4f}')


print("\nFinished Training...")

In [ ]:
plt.figure(figsize=(10, 6), dpi=300)  # High-resolution plot

plt.plot(train_losses, color='black', linewidth=1.5, label="Training loss")
plt.plot(eval_losses, color='red', linewidth=1.5, label="Validation loss")
plt.title("Training Loss and Validation Loss", fontsize=12)
plt.grid(True, linestyle='--', linewidth=0.5)
plt.xlabel("Epoch", fontsize=10)
plt.ylabel("MSE Loss", fontsize=10)
plt.legend(loc='upper right', fontsize=10)
# plt.gca().set_aspect('equal', adjustable='box')  # Maintain correct aspect ratio
# plt.savefig("wing_model_loss.pdf", format="pdf", dpi=300, bbox_inches="tight")

In [ ]:
if SAVE_TRAINED_MODEL:

    save_path =  model_path + '/models/wing/{}_LSTM_eMO_wingModel_static_lr{}_e{}_nL{}_numNN{}.pth'.format(date, learning_rate, num_epochs, num_layers, hidden_size)
    print("The model will be saved as the following:\n {}".format(save_path))

    torch.save(model.state_dict(), save_path)

    # Save the scalers
    joblib.dump(input_scaler, model_path+'/scalers/wing/{}_LSTM_eMO_wingModel_static_ipScaler_lr{}_e{}_nL{}_numNN{}.pkl'.format(date, learning_rate, num_epochs, num_layers, hidden_size))
    joblib.dump(output_scaler, model_path+'/scalers/wing/{}_LSTM_eMO_wingModel_static_opScaler_lr{}_e{}_nL{}_numNN{}.pkl'.format(date, learning_rate, num_epochs, num_layers, hidden_size))

In [ ]:
# Root directory where simulation subdirectories are stored
root_test_base = data_path + '/testing_data/'

for simulation_case in os.listdir(root_test_base):
    # Root directory where simulation subdirectories are stored
    root_dir_test_sim = root_test_base+simulation_case
  

    # Create the dataset with a subdirectory condition
    dataset_test = WingDataset(root_dir_test_sim, subdir_condition=subdir_condition)
    inputs_test, outputs_test = dataset_test[0:]

    # Assuming your input tensor is named `input_tensor`
    input_tensor_test = inputs_test.squeeze(1)  # Remove the singleton dimension at index 1
    print("Input shape:", input_tensor_test.shape)  # Should print: torch.Size([6, 145, 7])

    output_tensor_test = outputs_test.squeeze(1)
    print("Output shape:",output_tensor_test.shape)

    inputs_test_reshaped = input_tensor_test.reshape(-1, input_size)

    test_inputs_normalized = input_scaler.transform(inputs_test_reshaped.reshape(-1, input_size)).reshape(input_tensor_test.shape)

    test_inputs_tensor = torch.tensor(test_inputs_normalized, dtype=torch.float32).to(device)

    # Make predictions using the trained model
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        predicted_outputs, attention_weights = model(test_inputs_tensor)


    # Convert the predictions back to numpy and inverse scale the outputs
    predicted_outputs = predicted_outputs.cpu().detach().numpy()  # Convert tensor to numpy array
    predicted_outputs_original_scale = output_scaler.inverse_transform(predicted_outputs.reshape(-1, output_size))

    # Reshape the predictions to match the original sequence structure if needed
    predicted_outputs_original_scale = predicted_outputs_original_scale.reshape(input_tensor_test.shape[0], input_tensor_test.shape[1], output_size)
    predicted_outputs_original_scale = predicted_outputs_original_scale[0]


    cl_test_NN = predicted_outputs_original_scale[:,0]
    cd_test_NN = predicted_outputs_original_scale[:,1]

    # Load timesteps, CT and CQ from FLOWUnsteady simualtions
    time_steps = dataset_test.get_variable('time')

    cl_test_flowuns = dataset_test.get_variable('CL')
    cd_test_flowuns = dataset_test.get_variable('CD')

    if PLOT_RESULTS:
        # plt.figure()
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 2, 1)
        plt.plot(cl_test_NN, label = 'cl_NN')
        plt.plot((cl_test_flowuns[0]), label = 'cl_flowuns')
        plt.xlabel('Time [s]')
        plt.ylabel('Lift coefficient, $C_T$')
        plt.title(simulation_case)
        plt.legend()

        # plt.figure()
        plt.subplot(1, 2, 2)
        plt.plot(cd_test_NN, label = 'cd_NN')
        plt.plot(cd_test_flowuns[0], label = 'cd_flowuns')
        plt.xlabel('Time [s]')
        plt.ylabel('Drag coefficient, $C_Q$')
        plt.title(simulation_case)
        plt.legend()


## Evaluation

In [ ]:
from sklearn.metrics import r2_score

root_test_base = data_path + '/testing_data/'

def mape(y_true, y_pred):
    """Compute Mean Absolute Percentage Error (MAPE)"""
    mask = y_true != 0  # Avoid division by zero
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def relative_l2_norm(y_true, y_pred):
    """Compute Relative L2 Norm Error (ε)"""
    mask = y_true != 0  # Avoid division by zero
    numerator = np.linalg.norm(y_pred[mask] - y_true[mask], ord=2)  # ||pred - true||_2
    denominator = np.linalg.norm(y_true[mask], ord=2)  # ||true||_2
    return (numerator / denominator) * 100

# Initialize lists to store evaluation results
mape_cl_list, mape_cd_list = [], []
r2_cl_list, r2_cd_list = [], []
relative_l2_norm_cl_list, relative_l2_norm_cd_list = [], []

for simulation_case in os.listdir(root_test_base):
    root_dir_test_sim = root_test_base + simulation_case

    # Load dataset
    dataset_test = WingDataset(root_dir_test_sim, subdir_condition=subdir_condition)
    inputs_test, outputs_test = dataset_test[0:]

    input_tensor_test = inputs_test.squeeze(1)  # Remove singleton dimension
    output_tensor_test = outputs_test.squeeze(1)

    # Normalize inputs
    inputs_test_reshaped = input_tensor_test.reshape(-1, input_size)
    test_inputs_normalized = input_scaler.transform(inputs_test_reshaped).reshape(input_tensor_test.shape)
    test_inputs_tensor = torch.tensor(test_inputs_normalized, dtype=torch.float32).to(device)

    # Model prediction
    model.eval()
    with torch.no_grad():
        predicted_outputs, _ = model(test_inputs_tensor)  # Assuming model returns attention weights too

    # Convert predictions back to original scale
    predicted_outputs = predicted_outputs.cpu().detach().numpy()
    predicted_outputs_original_scale = output_scaler.inverse_transform(predicted_outputs.reshape(-1, output_size))
    predicted_outputs_original_scale = predicted_outputs_original_scale.reshape(input_tensor_test.shape[0], input_tensor_test.shape[1], output_size)
    predicted_outputs_original_scale = predicted_outputs_original_scale[0]  # Select first batch if needed

    # Extract Cl and Cd predictions
    cl_test_NN = predicted_outputs_original_scale[:, 0]
    cd_test_NN = predicted_outputs_original_scale[:, 1]

    # Load ground truth (FLOWUnsteady simulation)
    cl_test_flowuns = dataset_test.get_variable('CL')[0]  # Ensure correct shape
    cd_test_flowuns = dataset_test.get_variable('CD')[0]

    # Compute MAPE and R² Score
    mape_cl = mape(cl_test_flowuns, cl_test_NN)
    mape_cd = mape(cd_test_flowuns, cd_test_NN)
    r2_cl = r2_score(cl_test_flowuns, cl_test_NN)
    r2_cd = r2_score(cd_test_flowuns, cd_test_NN)
    relative_l2_norm_cl = relative_l2_norm(cl_test_flowuns, cl_test_NN)
    relative_l2_norm_cd = relative_l2_norm(cd_test_flowuns, cd_test_NN)


    # Store results
    mape_cl_list.append(mape_cl)
    mape_cd_list.append(mape_cd)
    r2_cl_list.append(r2_cl)
    r2_cd_list.append(r2_cd)
    relative_l2_norm_cl_list.append(relative_l2_norm_cl)
    relative_l2_norm_cd_list.append(relative_l2_norm_cd)

    # Print metrics for this simulation case
    print(f"Simulation Case: {simulation_case}")
    print(f"  MAPE Cl: {mape_cl:.2f}%, MAPE Cd: {mape_cd:.2f}%")
    print(f"  R² Cl: {r2_cl:.4f}, R² Cd: {r2_cd:.4f}")
    print(f"  ε Cl: {relative_l2_norm_cl:.2f}%, ε Cd: {relative_l2_norm_cd:.2f}%")

# Compute average metrics across all test cases
print("\nOverall Metrics Across All Test Cases:")
print(f"  Avg MAPE Cl: {np.mean(mape_cl_list):.2f}%, Avg MAPE Cd: {np.mean(mape_cd_list):.2f}%")
print(f"  Avg R² Cl: {np.mean(r2_cl_list):.4f}, Avg R² Cd: {np.mean(r2_cd_list):.4f}")
print(f"  Avg ε Cl: {np.mean(relative_l2_norm_cl_list):.2f}%, Avg ε Cd: {np.mean(relative_l2_norm_cd_list):.2f}%")